In [ ]:
import json
import gzip

import matplotlib.pyplot as plt  
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
import numpy as np


In [ ]:

def get_cols():
    cols = {
        "ovr0": [],
        "ovr1": [],
        "ovr2": [],
        "ovr3": [],
        "ovr4": [],
        "ovr5": [],
        "ovr6": [],
        "ovr7": [],
        "ovr8": [],
        "ovr9": [],
        "mov": [],
    }

    with gzip.GzipFile("data.json.gz", "r") as read_file:
        data = json.loads(read_file.read())

    def get_ovrs(tid, season):
        ovrs = []

        for p in data['players']:
            if tid in p['statsTids']:
                for ps in p['stats']:
                    if ps['season'] == season and ps['tid'] == tid:
                        found_ratings = False
                        for pr in p['ratings']:
                            if pr['season'] == season:
                                found_ratings = True
                                ovrs.append(pr['ovr'])
                                break
                        if not found_ratings:
                            raise Exception("No ratings found")
                        break
                    elif ps['season'] > season:
                        break

        ovrs.sort(reverse=True)

        return ovrs

    for t in data['teams']:
        tid = t['tid']
        for ts in t['stats']:
            if not ts['playoffs'] and ts['gp'] > 0:
                season = ts['season']
                mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
                cols['mov'].append(mov)

                ovrs = get_ovrs(tid, season)
                cols['ovr0'].append(ovrs[0])
                cols['ovr1'].append(ovrs[1])
                cols['ovr2'].append(ovrs[2])
                cols['ovr3'].append(ovrs[3])
                cols['ovr4'].append(ovrs[4])
                cols['ovr5'].append(ovrs[5])
                cols['ovr6'].append(ovrs[6])
                cols['ovr7'].append(ovrs[7])
                cols['ovr8'].append(ovrs[8])
                cols['ovr9'].append(ovrs[9])

    return cols

cols = get_cols()


In [ ]:
dataX

In [ ]:

dataset = pd.DataFrame(cols)

dataX = dataset[fit_cols]
#dataX = np.tile(np.arange(dataset[fit_cols].shape[1]),(dataset.shape[0],1))

fit_cols = ['ovr0', 'ovr1', 'ovr2', 'ovr3', 'ovr4', 'ovr5', 'ovr6', 'ovr7', 'ovr8', 'ovr9']
reg = sm.OLS(dataset['mov'],sm.add_constant(dataX))
res = reg.fit()

dataset['mov_predicted'] = res.predict()

#print('Intercept: \n', reg.params['const'])
#print('Coefficients: \n', reg.coef_)
print('r2: ', r2_score(dataset['mov'], dataset['mov_predicted']))

print(dataset)


# dataset.plot.hexbin(x='mov', y='mov_predicted', gridsize=20)
dataset.plot.scatter(x='mov', y='mov_predicted', alpha=0.2)
plt.xlabel('Actual MOV')  
plt.ylabel('Predicted MOV')  

plt.plot([-20, 20], [-20, 20])

plt.show()
res.summary()

In [ ]:
plt.plot(np.array(res.params[1:]))

In [ ]:
a,b = np.polyfit(np.arange(10),np.log(np.array(res.params[1:])),1)
plt.plot(np.array(res.params[1:]))
plt.plot(np.exp(np.arange(10)*a)*np.exp(b))
print(np.exp(b),a)

In [ ]:
np.log(dataset['mov'])

In [ ]:
np.array(dataX)

In [ ]:
import scipy.optimize as opt
from scipy.stats import pearsonr

In [ ]:
def best_fit_func_exp(x):
    return np.linalg.norm(dataX @ np.exp(x[0]*np.arange(10))*x[1]-x[2] - dataset['mov'])
def best_fit_func_lin(x):
    return np.linalg.norm(dataX @ (np.arange(10))*x[0]-x[1] - dataset['mov'])
def best_fit_func_quad(x):
    return np.linalg.norm(dataX @ ((np.arange(10)**2)*x[0] + np.arange(10) * x[1] + x[2]) - dataset['mov'])
best_fit_func_exp([a,np.exp(b),-res.params['const']])

In [ ]:
res2 = opt.minimize(best_fit_func_exp,[1,1,-100])
res2

In [ ]:
plt.plot(np.array(res.params[1:]))
plt.plot(np.exp(np.arange(10)*a)*np.exp(b))
plt.plot(np.exp(res2.x[0]*np.arange(10))*res2.x[1])

In [ ]:
from IPython.display import display, Math
display(Math('{:.2f} e^{{ {:.2f} x }}  + {:.2f}'.format(res2.x[1],res2.x[0],res2.x[2])))